In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data
import pandas as pd
from tqdm import trange
from src.models.TwoLayerLeakySNN import TwoLayerLeaky

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
# setting seeds
np.random.seed(445)
torch.manual_seed(445)

In [4]:
def get_accuracy(model, loader, pct_close):
    correct = 0
    total = 0
    model.eval()
    for x, labels in loader:
        x, labels = x.to(device), labels.to(device)
        output = model(x)
        pred = torch.round(output)
        correct += (torch.abs(pred-labels.view_as(pred)) < torch.abs(pct_close * labels.view_as(pred))).sum().item()
        total += x.shape[0]
    return correct / total

In [5]:
batch_size = 32
dtype = torch.float

## Data

In [6]:
data06 = pd.read_csv("../data/processed/target06.csv")

In [7]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(), torch.from_numpy(data06.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.7 * len(all_data)), round(0.2 * len(all_data)), round(0.1 * len(all_data))))

In [8]:
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

## Prepare model

In [9]:
inputs=8
hidden=1200
outputs=1

model = TwoLayerLeaky(inputs, hidden, outputs, 0.95, 20)
model.to(device)
print(model)

TwoLayerLeaky(
  (fc1): Linear(in_features=8, out_features=1200, bias=True)
  (lif1): Leaky()
  (fc2): Linear(in_features=1200, out_features=1, bias=True)
)


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999))
loss_fun = nn.MSELoss()
num_steps = 20

## Training

In [11]:
num_epochs = 21
loss_hist = []
test_loss_hist = []

# Outer training loop
for epoch in trange(num_epochs):
    for data, targets in train_loader:
        data = data.to(device)
        targets = targets.to(device)

    # forward pass
        model.train()
        preds = model(data).squeeze(dim=1)

        loss = loss_fun(preds, targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Store loss history for future plotting
        #loss_hist.append(loss.item())

    if epoch % 2 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}, acc: {get_accuracy(model, valid_loader, 0.2)}")

  5%|▍         | 1/21 [00:04<01:37,  4.86s/it]

Epoch: 0, loss: 4.35e+02, acc: 0.2277456647398844


 14%|█▍        | 3/21 [00:11<01:04,  3.56s/it]

Epoch: 2, loss: 2.32e+02, acc: 0.27398843930635836


 24%|██▍       | 5/21 [00:17<00:55,  3.44s/it]

...

Epoch: 16, loss: 1.2e+02, acc: 0.3468208092485549


 90%|█████████ | 19/21 [01:05<00:06,  3.39s/it]

Epoch: 18, loss: 78.4, acc: 0.34104046242774566


100%|██████████| 21/21 [01:12<00:00,  3.44s/it]

Epoch: 20, loss: 88.0, acc: 0.353757225433526


In [12]:
percent = 0.2
model.eval()
get_accuracy(model, test_loader, percent)

0.39710982658959537

In [13]:
targets

tensor([17.2600, 40.9700, 17.5000, 14.8100, 23.4800, 11.5400, 56.5900, 15.5000,
        18.7700,  6.9900, 27.5500, 17.2400, 12.7900, 13.4600,  5.5400,  8.7900,
        16.5200,  7.4300, 59.6500, 21.2600,  6.4300, 25.1300, 34.4500, 57.2000,
        37.3600, 24.4300, 15.4200, 21.3900, 13.5400,  5.9400,  9.3500, 53.5600],
       device='cuda:0')

In [14]:
preds

tensor([11.8930, 28.9670, 24.6963, 15.9994, 10.6815, 10.8053, 43.1768, 22.8555,
        22.3055,  8.9332, 23.5829, 15.4053, 15.3240, 33.3839,  8.9404, 19.2472,
        21.0731,  6.0380, 30.9477, 21.0582, 14.3365, 27.1071, 29.6288, 39.4087,
        25.7466, 15.3594, 12.2496, 14.6468, 13.0393, 11.1612, 11.9123, 45.1264],
       device='cuda:0', grad_fn=<SqueezeBackward1>)